In [13]:
# from google.colab import drive
# drive.mount("/content/drive")
# DRIVE_SRC = "/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/experiments/src"
# !rm -r -f ./sample_data
# !cp -r /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/.creds/.aws ~/
# !cp -r $DRIVE_SRC ./src

In [14]:
! pip install -qq sagemaker ultralytics

In [15]:
project_name = "/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/training-jobs"

# Detect

In [16]:
# experiment_name = "cricket-object-detect"

In [17]:
# from src.data.download import download
# config_path = download(experiment_name)

In [18]:
# from ultralytics import YOLO
# model = YOLO('yolov8n.pt')

# patience = 10
# epochs = 200
# model.train(data=config_path, epochs=epochs, patience=patience, project_name=project_name, experiment_name=experiment_name)

# Segment

In [19]:
# experiment_name = "batsmen-segmentation"

In [20]:
# from src.data.download import download
# ds_config_path = download(experiment_name)

In [21]:
# from ultralytics import YOLO
# model = YOLO('yolov8n-seg.pt')

# patience = 10
# epochs = 200

# model.train(data=ds_config_path, epochs=epochs, patience=patience, pretrained=True, project=project_name, name=experiment_name)